In [26]:
import cv2
import os
import random
import numpy as np

def generate_random_polygon(image, min_area):
    height, width, _ = image.shape
    
    while True:
        point_1 = [np.random.randint(0, width), np.random.randint(0, height)]
        point_2 = [np.random.randint(0, width), np.random.randint(0, height)]
        point_3 = [np.random.randint(0, width), np.random.randint(0, height)]
        polygon = np.array([point_1, point_2, point_3], np.int32)
        
        # Gets area of triangle
        area = 0.5 * abs((point_1[0]*(point_2[1] - point_3[1])) + 
                         (point_2[0]*(point_3[1] - point_1[1])) + 
                         (point_3[0]*(point_1[1] - point_2[1])))
        
        if area >= min_area:
            return polygon.reshape((-1, 1, 2))
        

def adjust_contrast_brightness(image):
    
    alpha = random.uniform(0.5, 1.5)
    beta = random.randint(-50, 50)

    image_float = image.astype(np.float32)
    
    adjusted_image = cv2.multiply(image_float, np.array([alpha]))
    adjusted_image = cv2.add(adjusted_image, beta)
    adjusted_image = np.clip(adjusted_image, 0, 255)
    adjusted_image = adjusted_image.astype(np.uint8)
    
    return adjusted_image


def add_triangle(image, num_shadows, min_area, transparency):

    image_with_shadow = np.copy(image)
    
    for _ in range(num_shadows):

        temp_img = np.copy(image)
        polygon = generate_random_polygon(image, min_area)
        shadow_color = (np.random.randint(0, 255), np.random.randint(0, 255), np.random.randint(0, 255))  
        
        cv2.fillPoly(temp_img, [polygon], shadow_color)
        cv2.addWeighted(image_with_shadow, transparency, temp_img, transparency, 0, image_with_shadow)
    
    return image_with_shadow

print(test)
def shear_image(image, shear_factor=0.0):

    rows, cols, _ = image.shape
    
    M = np.array([[1, shear_factor, 0],
                  [0, 1, 0]])
    
    sheared_img = cv2.warpAffine(image, M, (cols, rows))
    
    if shear_factor > 0:
        sheared_img = sheared_img[:, int(cols*shear_factor):]
    elif shear_factor < 0:
        sheared_img = sheared_img[:, :int(cols*(1+shear_factor))]
    
    return sheared_img


def process_image(image, min_area, num_shadows, shear_factor, poly_transparency):

    image = add_triangle(image, num_shadows=num_shadows, min_area=min_area, transparency=poly_transparency)
    image = adjust_contrast_brightness(image)
    image = shear_image(image, shear_factor=shear_factor)
    
    return image


backgrounds_dir = "bg_base_imgs"
backgrounds = os.listdir(backgrounds_dir)
selected_background_filename = random.choice(backgrounds)
print(selected_background_filename)
selected_background_path = os.path.join(backgrounds_dir, selected_background_filename)
background_img = cv2.imread(selected_background_path)

processed_image = process_image(background_img, 5000, 7, 0.2, 0.5)

display_size = (500, 500)
resized_processed_image = cv2.resize(processed_image, display_size)

cv2.imshow('Processed Image', resized_processed_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

pexels-jj-jordan-9304725.jpg
